# Example of a variational quantum circuit.

This is a very simple example of how to generate a custom circuit with a custom loss function.

In [127]:
from jqiskit.api import QuantumCircuit
import numpy as np

# Build a simple three bit quantum circuit.
n_qubits = 4
qc = QuantumCircuit(n_qubits)
qc.h(0)
qc.h(1)
qc.cx(0, 1)
# Some weird gates based on a phase shift gate.
qc.parametric('[[1., 0.,],[0., exp(1.0j * global_0)]]', 0)
qc.parametric('[[exp(-1.0j * global_1), 0.,],[0., exp(-1.0j * global_0)]]', 1)
qc.sqrtnot(0)
qc.sqrtnot(1)
qc.cswap(0, 1, 2)

print(qc)

Hadamard: (0,)
Hadamard: (1,)
CX: (0, 1)
Parametric: (0,)
Parametric: (1,)
SQRTNOT: (0,)
SQRTNOT: (1,)
CSWAP: (0, 1, 2)


In [130]:
def cost_function(global_0, global_1):
    """Simple cost function to put into the black-box optimizer."""
    counts, _ = qc.measure(num_shots=100, feed_dict={'global_0': global_0, 'global_1': global_1})
    values = list(counts.values())
    return np.mean(values[0] / np.sum(values) - 0.1)**2

In [131]:
from scipy.optimize import minimize

x0 = [np.pi / 2, np.pi / 2]
res = minimize(lambda x_in: cost_function(x_in[0], x_in[1]),
               x0, method='powell',
               bounds=((0.0, 2*np.pi), (0.0, 2 * np.pi)),
               tol=1e-5)
print(res)

{'0000': 42, '0100': 58}
{'0100': 72, '1010': 10, '1000': 2, '0000': 16}
{'1010': 80, '1000': 9, '0100': 11}
{'0100': 50, '0000': 50}
{'0000': 68, '0100': 23, '1010': 1, '1000': 8}
{'0000': 46, '0100': 54}
{'0000': 35, '0100': 63, '1010': 2}
{'0100': 64, '0000': 25, '1010': 6, '1000': 5}
{'0000': 38, '0100': 61, '1010': 1}
{'0000': 35, '0100': 61, '1000': 2, '1010': 2}
{'1010': 3, '0000': 32, '0100': 64, '1000': 1}
{'0000': 29, '0100': 70, '1010': 1}
{'0000': 24, '0100': 72, '1000': 3, '1010': 1}
{'0100': 65, '0000': 33, '1000': 1, '1010': 1}
{'0100': 60, '0000': 33, '1010': 6, '1000': 1}
{'0000': 70, '0100': 27, '1000': 3}
{'0000': 94, '1000': 4, '0100': 2}
{'0100': 73, '0000': 21, '1010': 5, '1000': 1}
{'0000': 65, '0100': 31, '1010': 3, '1000': 1}
{'0100': 54, '0000': 44, '1000': 2}
{'0100': 59, '0000': 39, '1010': 1, '1000': 1}
{'0100': 39, '0000': 60, '1000': 1}
{'0100': 56, '0000': 43, '1000': 1}
{'0000': 58, '0100': 37, '1010': 5}
{'0000': 40, '0100': 53, '1010': 6, '1000': 1}
{